In [12]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo

project_root = Path.cwd().parent
print(f"Project root: {project_root}")

Project root: /Users/david/Code/msc


In [13]:
# User input: run folder name 
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

params_path = run_folder / "sncosmo_parameters.csv"

df = pd.read_csv(params_path)
print(f"Loaded {len(df)} object(s) from {params_path}")
df.head()

Enter the run folder name: test4
Loaded 43 object(s) from /Users/david/Code/msc/runs/test4/sncosmo_parameters.csv


,object_id,chisq,ndof,z,t0,x0,x1,c,mwebv,t0_err,x0_err,x1_err,c_err
0,ZTF19aailqir,1.707646,7,0.05434,58529.181776,0.000451,0.139796,0.300000,0.024839,0.539645,0.000021,0.731697,0.413524
1,ZTF19aailqli,86.067663,19,0.05800,58535.586380,0.001421,0.197938,-0.039584,0.013871,0.240893,0.000094,0.368545,0.047733
2,ZTF19aailteb,16.736875,8,0.07800,58524.495481,0.000763,2.999994,-0.036656,0.014516,0.835964,0.000040,0.974345,0.053904
3,ZTF19aailuvj,4.448203,14,0.07100,58536.547675,0.000903,0.470430,-0.094157,0.021290,0.313551,0.000059,0.422184,0.047706
4,ZTF19aajwdqt,16.834560,13,0.08800,58538.834603,0.000837,3.000000,0.029961,0.046129,0.333009,0.000039,0.171656,0.046427


In [ ]:
C_KM_S = 299792.458  # speed of light in km/s

model = sncosmo.Model(source="salt2")
m_B_list = [] # this is the apparent magnitude at the peak of the light curve
mu_list = [] # this is the distance modulus at the peak of the light curve
d_L_Mpc_list = [] # this is the distance in megaparsecs at the peak of the light curve
H0_list = [] # this is the Hubble constant at the peak of the light curve

for _, row in df.iterrows():
    model.set(z=row["z"], t0=row["t0"], x0=row["x0"], x1=row["x1"], c=row["c"])
    # Peak apparent and absolute B magnitude from the fitted model
    m_B = model.source_peakmag("bessellb", "ab")
    M_B = model.source_peakabsmag("bessellb", "ab")
    mu_obs = m_B - M_B
    m_B_list.append(m_B)
    mu_list.append(mu_obs)
    # d_L in parsecs: mu = 5*log10(d_L/10pc) => d_L = 10^(mu/5 + 1) pc
    d_L_pc = 10 ** (mu_obs / 5.0 + 1.0)
    d_L_Mpc = d_L_pc / 1e6
    d_L_Mpc_list.append(d_L_Mpc)
    # H0 = c*z/d_L (low z approximation), with c in km/s, d_L in Mpc
    H0_list.append(C_KM_S * row["z"] / d_L_Mpc)

out = df[["object_id", "z", "t0", "x0", "x1", "c"]].copy()
out["m_B"] = np.array(m_B_list)
out["mu"] = np.array(mu_list)
out["d_L_Mpc"] = np.array(d_L_Mpc_list)
out["H0"] = np.array(H0_list)

for idx, row in out.iterrows():
    # print("\nsncosmo results for object_id:", row["object_id"])
    for col in out.columns:
        print(f"{col}: {row[col]}")

out


sncosmo results for object_id: ZTF19aailqir
object_id: ZTF19aailqir
z: 0.05434
t0: 58529.18177580767
x0: 0.0004511855188841
x1: 0.1397963119913829
c: 0.2999997941048146
m_B: 18.85962332933726
mu: 36.9444863769085
d_L_Mpc: 244.84840280940185
H0: 66.53391233432403

sncosmo results for object_id: ZTF19aailqli
object_id: ZTF19aailqli
z: 0.058
t0: 58535.586380150125
x0: 0.0014209252989333
x1: 0.1979383223820422
c: -0.0395841672328324
m_B: 17.619959980895498
mu: 37.09178155378977
d_L_Mpc: 262.0331940804036
H0: 66.35786212133334

sncosmo results for object_id: ZTF19aailteb
object_id: ZTF19aailteb
z: 0.078
t0: 58524.495480555095
x0: 0.0007627690726022
x1: 2.999993570179901
c: -0.0366563049841429
m_B: 18.285023501451715
mu: 37.76603126044475
d_L_Mpc: 357.44255195855885
H0: 65.419776117509

sncosmo results for object_id: ZTF19aailuvj
object_id: ZTF19aailuvj
z: 0.071
t0: 58536.547674900576
x0: 0.0009028127949364
x1: 0.4704298004792315
c: -0.0941572039577951
m_B: 18.111802125746223
mu: 37.5511286

,object_id,z,t0,x0,x1,c,m_B,mu,d_L_Mpc,H0
0,ZTF19aailqir,0.05434,58529.181776,0.000451,0.139796,0.300000,18.859623,36.944486,244.848403,66.533912
1,ZTF19aailqli,0.05800,58535.586380,0.001421,0.197938,-0.039584,17.619960,37.091782,262.033194,66.357862
2,ZTF19aailteb,0.07800,58524.495481,0.000763,2.999994,-0.036656,18.285024,37.766031,357.442552,65.419776
3,ZTF19aailuvj,0.07100,58536.547675,0.000903,0.470430,-0.094157,18.111802,37.551129,323.761887,65.743577
4,ZTF19aajwdqt,0.08800,58538.834603,0.000837,3.000000,0.029961,18.184347,38.043106,406.088885,64.965423
5,ZTF19aajwhbc,0.10000,58536.914900,0.000563,-0.571901,-0.139275,18.626441,38.338575,465.280591,64.432616
6,ZTF19aajxhwa,0.03300,58539.506247,0.003617,0.403831,-0.061157,16.605209,35.827306,146.373089,67.588593
7,ZTF19aajxwnz,0.01500,58544.708295,0.020554,0.849131,0.042802,14.717422,34.085560,65.631438,68.517268
8,ZTF19aakiwxx,0.04500,58537.073989,0.002080,-1.004063,-0.063543,17.207787,36.520132,201.384645,66.989520
9,ZTF19aakiyly,0.07000,58539.562164,0.001059,0.666689,-0.072604,17.938632,37.518787,318.975519,65.790228


In [15]:
# Save to run folder
out_path = run_folder / "distances.csv"
out.to_csv(out_path, index=False)
print(f"Saved to {out_path}")
display(out)

Saved to /Users/david/Code/msc/runs/test4/distances.csv


,object_id,z,t0,x0,x1,c,m_B,mu,d_L_Mpc,H0
0,ZTF19aailqir,0.05434,58529.181776,0.000451,0.139796,0.300000,18.859623,36.944486,244.848403,66.533912
1,ZTF19aailqli,0.05800,58535.586380,0.001421,0.197938,-0.039584,17.619960,37.091782,262.033194,66.357862
2,ZTF19aailteb,0.07800,58524.495481,0.000763,2.999994,-0.036656,18.285024,37.766031,357.442552,65.419776
3,ZTF19aailuvj,0.07100,58536.547675,0.000903,0.470430,-0.094157,18.111802,37.551129,323.761887,65.743577
4,ZTF19aajwdqt,0.08800,58538.834603,0.000837,3.000000,0.029961,18.184347,38.043106,406.088885,64.965423
5,ZTF19aajwhbc,0.10000,58536.914900,0.000563,-0.571901,-0.139275,18.626441,38.338575,465.280591,64.432616
6,ZTF19aajxhwa,0.03300,58539.506247,0.003617,0.403831,-0.061157,16.605209,35.827306,146.373089,67.588593
7,ZTF19aajxwnz,0.01500,58544.708295,0.020554,0.849131,0.042802,14.717422,34.085560,65.631438,68.517268
8,ZTF19aakiwxx,0.04500,58537.073989,0.002080,-1.004063,-0.063543,17.207787,36.520132,201.384645,66.989520
9,ZTF19aakiyly,0.07000,58539.562164,0.001059,0.666689,-0.072604,17.938632,37.518787,318.975519,65.790228
